# SD201 Project 

## Dataset (from a Kaggle competition) : Instacart Market Basket Analysis

Link : https://www.kaggle.com/c/instacart-market-basket-analysis/data

Blog post about the competition : https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2

Key points from the dataset:

- 3M grocery store orders
- 200,000+ Instacart users
- 4 to 100 orders for each user, timestamped

“The Instacart Online Grocery Shopping Dataset 2017”, Accessed from https://www.instacart.com/datasets/grocery-shopping-2017 on 10/12/2021"

## Introduction

In this notebook, we seek to use association rule mining algorithm to make recommendations based on frequently bought-together items from the Instacart online store.

We will discuss the following algorithms :

- Apriori algorithm
- Frequent Pattern Growth Algorithm (FP-Growth)
- ECLAT algorithm

### Setup

In [1]:
# # Run cell if using Google Colab
# # Mount the private Google Drive folder to access the .csv files
# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [2]:
# just in case you are running on Google Colab, you may run into a problem later on if you do not upgrade the package
%pip install mlxtend --upgrade

Requirement already up-to-date: mlxtend in c:\users\remy\anaconda3\lib\site-packages (0.19.0)
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pyECLAT

In [4]:
'''Python librairies''' 

# Utility librairies
import pandas as pd
import scipy.stats as s
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

# Preprocessing and pipeline librairies
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

# Wrapper to convert regular classifiers to multi-label classifiers
from sklearn.multioutput import MultiOutputClassifier

# Classifiers that support multi-label output
from sklearn.neighbors import KNeighborsClassifier

# Association rule preprocessing librairies
from mlxtend.preprocessing import TransactionEncoder

# Association rule mining librairies
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns.fpgrowth import fpgrowth
from pyECLAT import ECLAT

# Metrics
from sklearn.metrics import jaccard_score

# Pretty charts
import seaborn as sns
sns.set_theme(style="ticks")

In [5]:
# Open the data
path_to_csv = './instacart/'

op_prior = pd.read_csv(path_to_csv + 'order_products__prior.csv')
orders   = pd.read_csv(path_to_csv + 'orders.csv')
products = pd.read_csv(path_to_csv + 'products.csv')

## Data cleaning

For association rule mining we only need `order_id` and the ordered products for each order. Since there are not any null entries in the carts data, we do not have to deal with `nan` values here.

There are some null entries in the `days_since_prior_order` that we have to deal with but only for multi-label classification (see `SD201-Instacart-MultiLabelClassification` notebook).

#### Formatting the data 

We cannot exploit our relational data directly: we need to perform merges using the keys in the data, and then perform an aggregation over the ordered products to get arrays of ordered products for each order.

Moreover, instead of keeping all the items (which poses memory problems when applying the mining algorithms), we can keep only the most frequent items according to what was done in EDA.

As we will see, this poses some limitations but gives preliminary insights for the Instacart platform.

In [6]:
threshold = 5e-4
order_count = len(op_prior)

# Create the DataFrame of ordered products with their frequencies
item_freq = op_prior.product_id.value_counts()
item_freq = pd.DataFrame(item_freq.reset_index())
item_freq.rename(columns={'product_id':'n_occ', 'index':'product_id'}, inplace= True)
item_freq['frequency'] = item_freq['n_occ']/order_count

# Compare the number of products before and after the drop
bf_size = len(item_freq)
item_freq = item_freq[item_freq.frequency>threshold]
af_size = len(item_freq)
print('Number of products before :', bf_size, 'after:', af_size)

Number of products before : 49677 after: 250


In [7]:
# Drop all rows with unfrequently bought products
op_prior = op_prior[op_prior.product_id.isin(item_freq.product_id)]

In [8]:
def arrange_data(op_data):
    '''
    Format the data so that to each order corresponds an array of products (the carts).
    op_data can be either op_train or op_prior.
    '''
    
    # Merge product information with order information
    data = op_data[['order_id', 'product_id']]
    # Merge with time information
    data = data.merge(orders[['order_id', 'order_dow', 'order_hour_of_day',
                      'days_since_prior_order']], on='order_id')
    # Merge with product names
    data = data.merge(products[['product_id', 'product_name']], on='product_id')
    
    # Aggregate the carts into arrays
    groupby_cols = ['order_id',
                    'order_dow',
                    'order_hour_of_day',
                    'days_since_prior_order']
    data = data.groupby(groupby_cols).aggregate(list)
    
    # Rename the product_id column to 'cart'
    data.rename(columns = {'product_id':'cart'}, inplace = True)
    data.rename(columns = {'product_name':'cart_names'}, inplace = True)
    
    # Reset the index that was changed by the aggregation
    data = data.reset_index()
    
    return data

In [9]:
# Create the DataFrame with aggregated carts for each order
data = arrange_data(op_prior)

In [10]:
data

,order_id,order_dow,order_hour_of_day,days_since_prior_order,cart,cart_names
0,2,5,9,8.0,"[33120, 28985, 17794]","[Organic Egg Whites, Michigan Organic Kale, Ca..."
1,3,5,17,12.0,"[33754, 24838, 21903, 46667, 17461]",[Total 2% with Strawberry Lowfat Greek Straine...
2,4,1,9,7.0,[25146],[Original Orange Juice]
3,5,6,16,9.0,"[13176, 27966, 23909, 6184, 47209]","[Bag of Organic Bananas, Organic Raspberries, ..."
4,7,2,14,30.0,"[34050, 46802]","[Orange Juice, Pineapple Chunks]"
...,...,...,...,...,...,...
2452184,3421076,5,13,4.0,"[28849, 19348]","[No Salt Added Black Beans, Fat Free Milk]"
2452185,3421078,5,11,7.0,"[24852, 37646, 41844, 40396]","[Banana, Organic Gala Apples, Honey Nut Cheeri..."
2452186,3421080,1,11,2.0,"[31717, 27845, 41950]","[Organic Cilantro, Organic Whole Milk, Organic..."
2452187,3421082,2,18,4.0,"[43352, 16797]","[Raspberries, Strawberries]"


## Association rule mining 

### Definitions and metrics 

We define some metrics for association mining :

**Support** :  An association rule having a support of x% means that there are x% of all the transactions in the database that follow this rule.

**Confidence** : An association rule having a confidence of x% means that x% of the customers who purchased some set of items (called the **antecedent**) also bought another set of items (the **consequent**).

### Apriori algorithm

The Apriori algorithm is highly inefficient for large datasets. However the Apriori property can be used to reduce the size of the set of products on which we apply the Apriori algorithm.

The Apriori property states that :

*All subsets of a frequent itemset must be frequent.*

It follows that:

*If an itemset is infrequent, all its supersets will be infrequent.*

The two algorithms below improve upon the Apriori algorithm and are suitable for large datasets.


### FP-Growth algorithm 

In [11]:
# Preprocess the transactions by one-hot-encoding them
te = TransactionEncoder()
te.fit(list(data.cart_names))
ohe_transactions = te.transform(list(data.cart_names))
ohe_transactions_df = pd.DataFrame(ohe_transactions, columns=te.columns_)
ohe_transactions_df

,100% Raw Coconut Water,100% Recycled Paper Towels,100% Whole Wheat Bread,2% Reduced Fat Milk,2% Reduced Fat Organic Milk,Air Chilled Organic Boneless Skinless Chicken Breasts,Apple Honeycrisp Organic,Asparagus,Asparation/Broccolini/Baby Broccoli,Baby Spinach,...,Unsweetened Vanilla Almond Milk,Vanilla Almond Breeze Almond Milk,Vanilla Skyr Nonfat Yogurt,Watermelon Chunks,Whipped Cream Cheese,White Corn,Whole Milk,Yellow Bell Pepper,Yellow Onions,"YoKids Squeezers Organic Low-Fat Yogurt, Strawberry"
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2452184,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2452185,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2452186,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2452187,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


We define the support of an itemset (set of products) as the number of occurences of said itemset in the list of transactions. However the librairy we use for FP-Growth uses a percentage for the support so we need to do the conversion.

The minimum support is the threshold of support for which we consider an itemset. We can fine tune this parameter by hand.

In [12]:
# Define the minimum support
# Here we choose that there must be at least 10000 transactions  
# among the 2,6M in which the itemset is present
min_support = 10000/len(data.cart_names) 
print('Minimum support:', min_support)

# Compute the frequent itemsets
frequent_itemsets = fpgrowth(ohe_transactions_df, min_support=min_support, use_colnames = True)
frequent_itemsets

Minimum support: 0.004077989094641563


,support,itemsets
0,0.027749,(Carrots)
1,0.025538,(Michigan Organic Kale)
2,0.007456,(Organic Egg Whites)
3,0.092559,(Organic Baby Spinach)
4,0.019939,(Organic Ginger Root)
...,...,...
474,0.005861,"(Organic Large Extra Fancy Fuji Apple, Organic..."
475,0.005057,"(Organic Large Extra Fancy Fuji Apple, Banana)"
476,0.005228,"(Organic Large Extra Fancy Fuji Apple, Organic..."
477,0.005975,"(Half & Half, Banana)"


In [13]:
# Compute association rules
# We tune the min_threshold so as not to get too many results,
# only the most relevant ones in terms of confidence
conf_thresh=0.20
fpg_a_rules = association_rules(frequent_itemsets, metric="confidence",
                                min_threshold=conf_thresh)
fpg_a_rules.sort_values(by='confidence', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
19,"(Organic Raspberries, Organic Hass Avocado)",(Bag of Organic Bananas),0.010082,0.146927,0.004489,0.445294,3.030721,0.003008,1.537883
97,(Bartlett Pears),(Banana),0.013553,0.180668,0.005239,0.386550,2.139567,0.002790,1.335615
48,(Organic Fuji Apple),(Banana),0.034337,0.180668,0.013057,0.380264,2.104773,0.006854,1.322067
15,"(Organic Hass Avocado, Organic Strawberries)",(Bag of Organic Bananas),0.015757,0.146927,0.005867,0.372328,2.534106,0.003552,1.359106
30,(Honeycrisp Apple),(Banana),0.030556,0.180668,0.010905,0.356898,1.975440,0.005385,1.274032
...,...,...,...,...,...,...,...,...,...
64,(Organic Garlic),(Organic Yellow Onion),0.042071,0.043879,0.008554,0.203325,4.633786,0.006708,1.200139
81,(Organic Peeled Whole Baby Carrots),(Bag of Organic Bananas),0.021020,0.146927,0.004273,0.203283,1.383565,0.001185,1.070735
95,(Organic Granny Smith Apple),(Banana),0.022708,0.180668,0.004605,0.202801,1.122512,0.000503,1.027765
28,(Organic Cilantro),(Bag of Organic Bananas),0.026481,0.146927,0.005358,0.202322,1.377029,0.001467,1.069446


In [14]:
# Free the RAM for Google Colab
ohe_transactions_df = None
frequent_itemsets = None

We do get coherent association rules, for example someone who orders organic products is likely to order organic bananas as well. Therefore Instacart could suggest organic bananas to someone who already has organic products in their cart.

However since the initial dataset of transactions is mainly about fresh fruits and vegetables (as we have filtered them this way to get the most ordered products), we only have association rules 
for those products.

Nonetheless, we could easily solve this problem by filtering according to the department of each products, so as to make association rules for each department.

### ECLAT Algorithm

Unfortunateley the ECLAT algorithm crashes for all combinations of parameters on Google Colab...

In [15]:
# We are not interested in individual products
# We set the minimum itemset size to 2
min_n_products = 2

# Define the minimum support
# Here we choose that there must be n transactions among the 2,6M in which
# the itemset is present
n=26000
min_support = 26000/len(data.cart_names)

# There is no real limit to the size of association rules
# However we set it to 2 for memory reasons
max_length = 5

In [16]:
# Convert input data to the right format
ECLAT_transactions = pd.DataFrame(list(data.cart_names))

In [17]:
# # Crashes here
# eclat = ECLAT(data=ECLAT_transactions, verbose=True)

# # fit the algorithm
# rule_indices, rule_supports = my_eclat.fit(min_support=min_support,
#                                            min_combination=min_n_products,
#                                            max_combination=max_length)

In [18]:
# print(rule_supports)

## Combining the two methods

### Defining the model (run all)



#### Model and pipeline definition (k=3)

Same as before.

In [19]:
numerical_cols = ['order_dow', 'order_hour_of_day', 'days_since_prior_order']

# Impute the average over all orders
avg_imp = SimpleImputer(missing_values=np.nan, strategy='mean')

# Imputing the average for a given client in a pipeline necessitates writing a custom imputer.
# This is optional and will be done if there is enough time.

# Min-max normalization 
mm_scaler = MinMaxScaler()
std_scaler = StandardScaler()

# Define the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', avg_imp, numerical_cols),
        ('norm', std_scaler, numerical_cols)
    ])

In [20]:
# Create the model

# kNN Classifier
# Used GridSearchCV to tune k to 3
kNN_model = KNeighborsClassifier(n_neighbors=3)
multi_kNN_model = MultiOutputClassifier(kNN_model, n_jobs=-1)

In [21]:
# Make the pipeline

# kNN Classifier
kNN_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('kNN model', multi_kNN_model)
                             ],
                        verbose=True)

### Combining all the steps

We can now combine our KNN tuned model with the association rules from the FP-Growth algorithm and see whether the jaccard score improves or not.

In [22]:
# Create the training data
train_data = data[['order_id', 'order_dow', 'order_hour_of_day',
                   'days_since_prior_order', 'cart']]

In [23]:
# Fraction of data to keep
frac = 1/10
# Testing size fraci
t_size=0.30

# Arrange training and testing data

features = ['order_dow', 'order_hour_of_day', 'days_since_prior_order']
target = 'cart'

# Take more data for training
train_sample = train_data.sample(axis=0, frac=frac)

# Define target and features
X = train_sample[features]
y = train_sample[target]

# Fit the MultiLabelBinarizer with sparse output
mlb = MultiLabelBinarizer()
mlb.fit(y)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=t_size)

# Convert the target data into binary matrix
y_train_bm = mlb.transform(y_train)
y_test_bm = mlb.transform(y_test)

In [24]:
# Fit the model for k=3
kNN_pipeline.fit(X_train, y_train_bm)

[Pipeline] ...... (step 1 of 2) Processing preprocessor, total=   0.1s
[Pipeline] ......... (step 2 of 2) Processing kNN model, total= 3.2min


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', SimpleImputer(),
                                                  ['order_dow',
                                                   'order_hour_of_day',
                                                   'days_since_prior_order']),
                                                 ('norm', StandardScaler(),
                                                  ['order_dow',
                                                   'order_hour_of_day',
                                                   'days_since_prior_order'])])),
                ('kNN model',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(n_neighbors=3),
                                       n_jobs=-1))],
         verbose=True)

In [25]:
# Predict on test set
y_pred_bm = kNN_pipeline.predict(X_test)

In [26]:
# Get baseline score
print('Raw Jaccard score:', jaccard_score(y_pred_bm, y_test_bm, average='samples'))

Raw Jaccard score: 0.010135476514248077


We define a few functions for getting the jaccard score only using non-empty carts. This should give us a better baseline score since the score calculation is made by divising by the total number of carts. 

Moreover this new score reflets the performance of our model better since there are a lot of items that our model cannot predict (we have only kept the most popular items on Instacart, and the test set contains carts with any item). 

In [36]:
def convert_to_carts(y_pred):
    """Convert back binary matrix prediction outputs to product id arrays"""
    arr = mlb.inverse_transform(y_pred)
    carts = [[] for i in range (len(arr))]
    for i in range(len(arr)):
        for id in arr[i]:
            carts[i].append(id)
    return carts

def get_non_empty_carts(carts_bm):
    """Returns a set of non-empty carts in binary matrix form
       from a set of binary matrices."""
    y_pred = convert_to_carts(carts_bm)
    y_pred_no_e = []
    idx = 0
    non_empty_idx = []
    
    # Build array with only non-empty carts
    # And save the indices of these carts
    for cart in y_pred:
        if cart != []:
            y_pred_no_e.append(cart)
            non_empty_idx.append(idx)
        idx+=1
        
    # Transform the previous array to binary matrix format
    non_e_carts_bm = mlb.transform(y_pred_no_e)
    
    return non_e_carts_bm, non_empty_idx
    
def jaccard_score_non_empty(y_pred_bm,y_test_bm):
    """Get the jaccard score for non empty carts only"""
    non_empty_pred_bm, non_empty_idx = get_non_empty_carts(y_pred_bm)    
    
    
    return jaccard_score(non_empty_pred_bm,
           y_test_bm[non_empty_idx],
           average='samples')

In [28]:
# Get baseline score for non-empty carts only
print('Jaccard score with non-empty carts:', jaccard_score_non_empty(y_pred_bm,y_test_bm))

Jaccard score with non-empty carts: 0.02841672568494127


In [29]:
# Get the association rules
a_rules = fpg_a_rules

In [30]:
# Set product name as index
prod_n_idx = products.set_index('product_name')

# Set product_id as index
prod_id_idx = products.set_index('product_id')

In [31]:
# Translate association rules to product_id
ante_id = [[] for i in range(len(a_rules))]
conse_id= [[] for i in range(len(a_rules))]

for antecedent_n in range(len(a_rules)):
    for product in a_rules.antecedents[antecedent_n]:
        ante_id[antecedent_n].append(prod_n_idx.loc[product].product_id)

for consequent_n in range(len(a_rules)):
    for product in a_rules.consequents[consequent_n]:
        conse_id[consequent_n].append(prod_n_idx.loc[product].product_id)

In [32]:
# Get predicts carts into array format (instead of tuples)
y_pred = mlb.inverse_transform(y_pred_bm)
y_pred_arr = [[] for i in range(len(y_pred))]
for cart_num in range(len(y_pred)):
    y_pred_arr[cart_num] = list(y_pred[cart_num])

In [33]:
# 1 to print when an item is added to a cart
# 0 to not print anything
comp_cart_verbose = 0

# Create completed carts
for cart_num in range(len(y_pred)):
    for ante_num in range(len(ante_id)):
        
    # Check if a cart is exactly equal to an antecedent
        if y_pred_arr[cart_num] == ante_id[ante_num]:
            for conse in conse_id[ante_num]:
                
                # Check if consequent is already in the cart
                # If not add it to the cart
                if conse not in y_pred_arr[cart_num]:
                    y_pred_arr[cart_num].append(conse)
                    
                    # Print info if verbose is set to 1
                    if comp_cart_verbose:
                        print('Added item:', prod_id_idx.loc[conse].product_name,
                      'in cart:', cart_num)

In [34]:
# Get new score
y_pred_arr_bm = mlb.transform(y_pred_arr)
print('New raw jaccard score:', jaccard_score(y_pred_arr_bm, y_test_bm, average='samples'))

New raw jaccard score: 0.014170038695429018


In [35]:
# Get new score for non-empty carts only
jaccard_score_non_empty(y_pred_arr_bm,y_test_bm)

Jaccard score with non-empty carts: 0.039728383957770154


We have successfully improved upon our ML model!

In [37]:
# Relative improvement in percentage
abs(jaccard_score_non_empty(y_pred_arr_bm,y_test_bm)-jaccard_score_non_empty(y_pred_bm,y_test_bm))/jaccard_score_non_empty(y_pred_bm,y_test_bm)

0.3980633940110564